In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# === Create output directory ===
os.makedirs("BRCA_ML_Outputs", exist_ok=True)

# === Load expression and metadata ===
expr = pd.read_csv("BRCA_VST_Normalized_Matrix.csv", index_col=0)
meta = pd.read_csv("BRCA_Metadata_Final.csv", index_col=0)

# === Manually define the top 4 genes ===
selected_genes = ['FOXO4', 'EGFR', 'FGF2', 'CDKN2A']

# === Filter expression matrix ===
expr.index = expr.index.str.upper()
expr = expr.loc[expr.index.isin(selected_genes)]
expr = expr.loc[selected_genes]  # preserve order
X = expr.T

# === Prepare class labels ===
meta['Group'] = meta['sample_type'].replace({
    "Solid Tissue Normal": "Normal",
    "Primary Tumor": "Tumor"
})
y = meta.loc[X.index, 'Group'].values
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # Normal = 0, Tumor = 1

# === Scale features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Define classification models ===
models = {
    'RF': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, kernel='rbf', random_state=42),
    'XGB': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'ADA': AdaBoostClassifier(random_state=42),
    'GBM': GradientBoostingClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'LOGIT': LogisticRegressionCV(max_iter=1000, random_state=42),
    'TREE': DecisionTreeClassifier(random_state=42),
    'NB': GaussianNB(),
    'ANN': MLPClassifier(hidden_layer_sizes=(50,), max_iter=1000, random_state=42),
    'LASSO': LogisticRegressionCV(
        penalty='l1',
        solver='liblinear',
        cv=5,
        random_state=1,
        max_iter=1000,
        tol=0.0001,
        refit=True,
        class_weight="balanced"
    )
}

# === Bootstrapping loop ===
n_bootstraps = 100
final_results = []

for model_name, model in models.items():
    print(f"🔁 Bootstrapping: {model_name}")
    sss = StratifiedShuffleSplit(n_splits=n_bootstraps, test_size=0.2, random_state=42)

    for train_index, test_index in sss.split(X_scaled, y_encoded):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y_encoded[train_index], y_encoded[test_index]

        pipeline = Pipeline([
            ('smote', SMOTE(random_state=42)),
            ('clf', model)
        ])
        pipeline.fit(X_train, y_train)

        for split, X_set, y_true in [('Train', X_train, y_train), ('Test', X_test, y_test)]:
            y_pred = pipeline.predict(X_set)
            y_proba = pipeline.predict_proba(X_set)[:, 1]

            acc = accuracy_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
            prec = precision_score(y_true, y_pred)
            rec = recall_score(y_true, y_pred)
            auc = roc_auc_score(y_true, y_proba)
            cm = confusion_matrix(y_true, y_pred)

            if cm.shape == (2, 2):
                TN, FP, FN, TP = cm.ravel()
                specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
                sensitivity = TP / (TP + FN) if (TP + FN) != 0 else 0
            else:
                specificity = sensitivity = 0

            final_results.append({
                'Model': model_name,
                'Split': split,
                'Accuracy': acc,
                'F1 Score': f1,
                'Precision': prec,
                'Recall': rec,
                'ROC AUC': auc,
                'Sensitivity': sensitivity,
                'Specificity': specificity
            })

# === Save performance results ===
df_results = pd.DataFrame(final_results)
agg_results = df_results.groupby(['Model', 'Split']).agg(['mean', 'std']).reset_index()
agg_results.columns = ['_'.join(col).strip('_') for col in agg_results.columns.values]
agg_results.to_excel("BRCA_ML_Outputs/ML_Performance_BRCA_4GenePanel_Bootstrap_TrainTest.xlsx", index=False)

print("✅ Bootstrapping complete. Results saved to Excel.")

🔁 Bootstrapping: RF
🔁 Bootstrapping: SVM
🔁 Bootstrapping: XGB


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:42:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:42:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:42:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:42:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:42:53] WARNING: /w

🔁 Bootstrapping: ADA
🔁 Bootstrapping: GBM
🔁 Bootstrapping: KNN
🔁 Bootstrapping: LOGIT
🔁 Bootstrapping: TREE
🔁 Bootstrapping: NB
🔁 Bootstrapping: ANN
🔁 Bootstrapping: LASSO
✅ Bootstrapping complete. Results saved to Excel.


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# =========================
# Setup
# =========================
os.makedirs("BRCA_ML_Outputs", exist_ok=True)

# Load expression (rows=genes, cols=samples) and metadata (rows=samples)
expr = pd.read_csv("BRCA_VST_Normalized_Matrix.csv", index_col=0)
meta = pd.read_csv("BRCA_Metadata_Final.csv", index_col=0)

# Harmonize gene symbols and align samples
expr.index = expr.index.str.upper()
X_all = expr.T
common_samples = X_all.index.intersection(meta.index)
X_all = X_all.loc[common_samples]
meta = meta.loc[common_samples].copy()

# Labels
meta['Group'] = meta['sample_type'].replace({
    "Solid Tissue Normal": "Normal",
    "Primary Tumor": "Tumor"
})
y = meta['Group'].values
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # Normal=0, Tumor=1

# Load DEG list (flexible column handling)
deg_path = "DESeq2_BRCA_DEGs_filtered.csv"
deg_df = pd.read_csv(deg_path)
deg_cols_try = ['gene', 'Gene', 'SYMBOL', 'symbol', 'hgnc_symbol', 'HGNC', 'ensembl_symbol', deg_df.columns[0]]
deg_col = next(c for c in deg_cols_try if c in deg_df.columns)
deg_genes = deg_df[deg_col].astype(str).str.upper().unique().tolist()

# Intersect DEGs with expression genes
deg_genes_in_expr = [g for g in deg_genes if g in expr.index]
X_deg = expr.loc[deg_genes_in_expr].T
X_deg = X_deg.loc[common_samples]

# =========================
# Models
# =========================
models = {
    'RF': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    'SVM': SVC(probability=True, kernel='rbf', random_state=42),  # can be slow on 40k+ features
    'XGB': XGBClassifier(
        tree_method='hist',
        eval_metric='logloss',
        random_state=42,
        n_estimators=300,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8
    ),
    'ADA': AdaBoostClassifier(random_state=42),
    'GBM': GradientBoostingClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'LOGIT': LogisticRegressionCV(cv=5, max_iter=2000, random_state=42, n_jobs=-1),
    'TREE': DecisionTreeClassifier(random_state=42),
    'NB': GaussianNB(),
    'ANN': MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42, early_stopping=True),
    'LASSO': LogisticRegressionCV(
        penalty='l1', solver='liblinear', cv=5,
        random_state=42, max_iter=2000, class_weight='balanced'
    )
}

# =========================
# Runner
# =========================
def run_experiment(X, y_enc, feature_set_name, n_bootstraps=100):
    results = []
    print(f"\n🧪 Running feature set: {feature_set_name} | n_features={X.shape[1]:,} | n_samples={X.shape[0]:,}")
    sss = StratifiedShuffleSplit(n_splits=n_bootstraps, test_size=0.2, random_state=42)

    for model_name, model in models.items():
        print(f"  🔁 {feature_set_name} → Bootstrapping: {model_name}")
        for train_idx, test_idx in sss.split(X.values, y_enc):
            X_train, X_test = X.values[train_idx], X.values[test_idx]
            y_train, y_test = y_enc[train_idx], y_enc[test_idx]

            pipe = Pipeline(steps=[
                ('scaler', StandardScaler(with_mean=True, with_std=True)),
                ('smote', SMOTE(random_state=42)),
                ('clf', model)
            ])

            pipe.fit(X_train, y_train)

            for split, X_set, y_true in [('Train', X_train, y_train), ('Test', X_test, y_test)]:
                y_pred = pipe.predict(X_set)
                # Robust probability handling
                try:
                    y_proba = pipe.predict_proba(X_set)[:, 1]
                except Exception:
                    if hasattr(pipe.named_steps['clf'], "decision_function"):
                        scores = pipe.named_steps['clf'].decision_function(
                            pipe.named_steps['scaler'].transform(X_set)
                        )
                        s_min, s_max = scores.min(), scores.max()
                        y_proba = (scores - s_min) / (s_max - s_min + 1e-12)
                    else:
                        y_proba = y_pred.astype(float)

                acc = accuracy_score(y_true, y_pred)
                f1 = f1_score(y_true, y_pred)
                prec = precision_score(y_true, y_pred)
                rec = recall_score(y_true, y_pred)
                auc = roc_auc_score(y_true, y_proba)

                cm = confusion_matrix(y_true, y_pred)
                if cm.shape == (2, 2):
                    TN, FP, FN, TP = cm.ravel()
                    specificity = TN / (TN + FP) if (TN + FP) else 0.0
                    sensitivity = TP / (TP + FN) if (TP + FN) else 0.0
                else:
                    specificity = sensitivity = 0.0

                results.append({
                    'FeatureSet': feature_set_name,
                    'Model': model_name,
                    'Split': split,
                    'Accuracy': acc,
                    'F1 Score': f1,
                    'Precision': prec,
                    'Recall': rec,
                    'ROC AUC': auc,
                    'Sensitivity': sensitivity,
                    'Specificity': specificity
                })
    return pd.DataFrame(results)

# =========================
# Execute: ALL genes and DEGs
# =========================
df_all = run_experiment(X_all, y_encoded, feature_set_name="ALL_GENES", n_bootstraps=100)
df_deg = run_experiment(X_deg, y_encoded, feature_set_name="DEG_ONLY", n_bootstraps=100)

# Aggregate + Save
df_combined = pd.concat([df_all, df_deg], ignore_index=True)
agg = (df_combined
       .groupby(['FeatureSet', 'Model', 'Split'])
       .agg(['mean', 'std'])
       .reset_index())
agg.columns = ['_'.join(col).strip('_') for col in agg.columns.values]

df_all.to_csv("BRCA_ML_Outputs/Raw_Bootstrap_Scores_ALL_GENES.csv", index=False)
df_deg.to_csv("BRCA_ML_Outputs/Raw_Bootstrap_Scores_DEG_ONLY.csv", index=False)
agg.to_excel("BRCA_ML_Outputs/ML_Performance_BRCA_ALLvsDEG_Bootstrap_TrainTest.xlsx", index=False)

print("\n✅ Done. Saved:")
print(" - BRCA_ML_Outputs/Raw_Bootstrap_Scores_ALL_GENES.csv")
print(" - BRCA_ML_Outputs/Raw_Bootstrap_Scores_DEG_ONLY.csv")
print(" - BRCA_ML_Outputs/ML_Performance_BRCA_ALLvsDEG_Bootstrap_TrainTest.xlsx")


🧪 Running feature set: ALL_GENES | n_features=40,967 | n_samples=1,224
  🔁 ALL_GENES → Bootstrapping: RF
  🔁 ALL_GENES → Bootstrapping: SVM
  🔁 ALL_GENES → Bootstrapping: XGB
  🔁 ALL_GENES → Bootstrapping: ADA


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score,
                             roc_auc_score, confusion_matrix)
from sklearn.linear_model import LogisticRegressionCV
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# =========================
# Paths / config
# =========================
OUTDIR = "BRCA_ML_Outputs"
os.makedirs(OUTDIR, exist_ok=True)

EXPR_PATH = "BRCA_VST_Normalized_Matrix.csv"   # rows=genes, cols=samples
META_PATH = "BRCA_Metadata_Final.csv"          # rows=samples
DEG_PATH  = "DESeq2_BRCA_DEGs_filtered.csv"    # contains a gene symbol column

# Grid and CV (tune here if runtime is long)
CS_GRID   = np.logspace(-3, 3, 7)   # e.g., [1e-3 ... 1e3]; shrink to 5 values if needed
CV_FOLDS  = 5                       # drop to 3 to speed up

# =========================
# Load expression & metadata
# =========================
expr = pd.read_csv(EXPR_PATH, index_col=0)
expr.index = expr.index.str.upper()

meta = pd.read_csv(META_PATH, index_col=0)
meta['Group'] = meta['sample_type'].replace({
    "Solid Tissue Normal": "Normal",
    "Primary Tumor": "Tumor"
})

# Align samples
X_all = expr.T
common_samples = X_all.index.intersection(meta.index)
X_all = X_all.loc[common_samples]
meta  = meta.loc[common_samples].copy()

# Encode labels AFTER alignment
le = LabelEncoder()
y  = le.fit_transform(meta['Group'].values)  # Normal=0, Tumor=1

# =========================
# Build DEG-only matrix
# =========================
deg_df = pd.read_csv(DEG_PATH)
deg_cols_try = ['gene', 'Gene', 'SYMBOL', 'symbol', 'hgnc_symbol', 'HGNC',
                'ensembl_symbol', deg_df.columns[0]]
deg_col = next(c for c in deg_cols_try if c in deg_df.columns)
deg_genes = deg_df[deg_col].astype(str).str.upper().unique().tolist()

deg_genes_in_expr = [g for g in deg_genes if g in expr.index]
X_deg = expr.loc[deg_genes_in_expr].T
X_deg = X_deg.loc[common_samples]

# =========================
# Runner (single split, liblinear)
# =========================
def run_lasso_liblinear_once(Xdf, y, feature_set_name, random_state=42):
    print(f"\n▶ {feature_set_name}: n_samples={Xdf.shape[0]:,}, n_features={Xdf.shape[1]:,}")

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)
    train_idx, test_idx = next(sss.split(Xdf.values, y))
    X_train, X_test = Xdf.values[train_idx], Xdf.values[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    train_ids = Xdf.index[train_idx].to_numpy()
    test_ids  = Xdf.index[test_idx].to_numpy()

    clf = LogisticRegressionCV(
        penalty='l1',
        solver='liblinear',        # <- uniform with your other analyses
        cv=CV_FOLDS,
        Cs=CS_GRID,
        scoring='roc_auc',
        max_iter=5000,
        n_jobs=-1,                 # parallelize CV across folds
        class_weight='balanced',
        refit=True,
        random_state=random_state
    )

    pipe = Pipeline(steps=[
        ('scaler', StandardScaler(with_mean=True, with_std=True)),
        ('smote',  SMOTE(random_state=random_state)),
        ('clf',    clf)
    ])

    pipe.fit(X_train, y_train)

    def eval_split(X_set, y_true, ids, split_name):
        y_pred  = pipe.predict(X_set)
        y_proba = pipe.predict_proba(X_set)[:, 1]
        acc = accuracy_score(y_true, y_pred)
        f1  = f1_score(y_true, y_pred)
        prec= precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_proba)
        cm  = confusion_matrix(y_true, y_pred)
        TN, FP, FN, TP = cm.ravel() if cm.shape == (2,2) else (0,0,0,0)
        spec = TN/(TN+FP) if (TN+FP) else 0.0
        sens = TP/(TP+FN) if (TP+FN) else 0.0

        metrics = pd.DataFrame([{
            'FeatureSet': feature_set_name,
            'Split': split_name,
            'Accuracy': acc,
            'F1 Score': f1,
            'Precision': prec,
            'Recall': rec,
            'ROC AUC': auc,
            'Sensitivity': sens,
            'Specificity': spec,
            'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN
        }])

        preds = pd.DataFrame({
            'Sample': ids,
            'true_class_(0/1)': y_true,
            'predicted_probability': y_proba,
            'predicted_label': y_pred
        })
        return metrics, preds

    m_train, pred_train = eval_split(X_train, y_train, train_ids, "Train")
    m_test,  pred_test  = eval_split(X_test,  y_test,  test_ids,  "Test")

    # Coefficients (on standardized features)
    coef = pipe.named_steps['clf'].coef_[0]
    coef_df = pd.DataFrame({'Feature': Xdf.columns, 'Coef': coef}).sort_values('Coef', ascending=False)

    # Save
    base = os.path.join(OUTDIR, f"LASSO_LIBLINEAR_{feature_set_name}")
    metrics_all = pd.concat([m_train, m_test], ignore_index=True)
    metrics_all.to_csv(base + "_Metrics.csv", index=False)
    pred_train.to_csv(base + "_Predictions_Train.csv", index=False)
    pred_test.to_csv(base + "_Predictions_Test.csv", index=False)
    coef_df.to_csv(base + "_Coefficients.csv", index=False)

    print(f"✓ Saved: {base}_Metrics.csv")
    print(f"✓ Saved: {base}_Predictions_Train.csv")
    print(f"✓ Saved: {base}_Predictions_Test.csv")
    print(f"✓ Saved: {base}_Coefficients.csv")
    return metrics_all, coef_df

# =========================
# Run
# =========================
metrics_all, coefs_all = run_lasso_liblinear_once(X_all, y, "ALL_GENES")
metrics_deg, coefs_deg = run_lasso_liblinear_once(X_deg, y, "DEG_ONLY")

summary = pd.concat([metrics_all, metrics_deg], ignore_index=True)
summary.to_excel(os.path.join(OUTDIR, "LASSO_Liblinear_AllVsDEG_Summary.xlsx"), index=False)
print("✓ Saved: BRCA_ML_Outputs/LASSO_Liblinear_AllVsDEG_Summary.xlsx")


▶ ALL_GENES: n_samples=1,224, n_features=40,967
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_ALL_GENES_Metrics.csv
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_ALL_GENES_Predictions_Train.csv
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_ALL_GENES_Predictions_Test.csv
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_ALL_GENES_Coefficients.csv

▶ DEG_ONLY: n_samples=1,224, n_features=5,737
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_DEG_ONLY_Metrics.csv
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_DEG_ONLY_Predictions_Train.csv
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_DEG_ONLY_Predictions_Test.csv
✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_DEG_ONLY_Coefficients.csv
✓ Saved: BRCA_ML_Outputs/LASSO_Liblinear_AllVsDEG_Summary.xlsx


In [ ]:
#without SMOTE & bootsraps (balanced and non balanced)
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix
)

# =========================
# Paths / config
# =========================
OUTDIR    = "BRCA_ML_Outputs"
EXPR_PATH = "BRCA_VST_Normalized_Matrix.csv"   # rows=genes, cols=samples
META_PATH = "BRCA_Metadata_Final.csv"          # rows=samples
DEG_PATH  = "DESeq2_BRCA_DEGs_filtered.csv"    # has a gene symbol column

os.makedirs(OUTDIR, exist_ok=True)

# CV & grid (tweak if you need to speed up)
CV_FOLDS  = 5
CS_GRID   = np.logspace(-3, 3, 7)  # [1e-3 ... 1e3]
TEST_SIZE = 0.20
RNG       = 42

# Run both weight modes to compare
CLASS_WEIGHT_MODES = ["balanced", "none"]   # "none" => class_weight=None

# =========================
# Load data & align samples
# =========================
expr = pd.read_csv(EXPR_PATH, index_col=0)
expr.index = expr.index.str.upper()

meta = pd.read_csv(META_PATH, index_col=0)
meta['Group'] = meta['sample_type'].replace({
    "Solid Tissue Normal": "Normal",
    "Primary Tumor": "Tumor"
})

# align (samples = rows, features = cols)
X_all = expr.T
common_samples = X_all.index.intersection(meta.index)
X_all = X_all.loc[common_samples]
meta  = meta.loc[common_samples].copy()

le = LabelEncoder()
y  = le.fit_transform(meta['Group'].values)  # Normal=0, Tumor=1

# =========================
# Build DEG-only matrix
# =========================
deg_df = pd.read_csv(DEG_PATH)
deg_cols_try = ['gene','Gene','SYMBOL','symbol','hgnc_symbol','HGNC','ensembl_symbol', deg_df.columns[0]]
deg_col = next(c for c in deg_cols_try if c in deg_df.columns)
deg_genes = deg_df[deg_col].astype(str).str.upper().unique().tolist()
deg_genes_in_expr = [g for g in deg_genes if g in expr.index]

X_deg = expr.loc[deg_genes_in_expr].T
X_deg = X_deg.loc[common_samples]

# =========================
# Helpers
# =========================
def eval_metrics(y_true, y_prob, y_pred):
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred)
    prec= precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)

    cm  = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = cm.ravel() if cm.shape==(2,2) else (0,0,0,0)

    sens_pos = TP/(TP+FN) if (TP+FN) else 0.0         # sensitivity for Tumor (pos=1)
    spec_pos = TN/(TN+FP) if (TN+FP) else 0.0         # specificity for Tumor (pos=1)
    # also report Normal-recall (pos=0) if you want
    rec_normal = recall_score(y_true, y_pred, pos_label=0)
    bal_acc = 0.5 * (sens_pos + spec_pos)

    return {
        'Accuracy': acc, 'F1 Score': f1, 'Precision': prec, 'Recall': rec,
        'ROC AUC': auc, 'Sensitivity(Tumor)': sens_pos, 'Specificity(Tumor)': spec_pos,
        'Recall(Normal)': rec_normal, 'Balanced Accuracy': bal_acc,
        'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN
    }

def run_lasso_once(Xdf, y, feature_set_name, class_weight_mode="balanced", random_state=RNG):
    print(f"\n▶ {feature_set_name} | class_weight={class_weight_mode} | n_samples={Xdf.shape[0]:,} | n_features={Xdf.shape[1]:,}")

    cw = 'balanced' if class_weight_mode == "balanced" else None

    # single stratified split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=random_state)
    tr_idx, te_idx = next(sss.split(Xdf.values, y))
    X_tr, X_te = Xdf.values[tr_idx], Xdf.values[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]
    tr_ids, te_ids = Xdf.index[tr_idx], Xdf.index[te_idx]

    # pipeline: Scale -> L1 logistic (liblinear)
    clf = LogisticRegressionCV(
        penalty='l1',
        solver='liblinear',     # keep uniform with your other analyses
        cv=CV_FOLDS,
        Cs=CS_GRID,
        scoring='roc_auc',
        max_iter=5000,
        n_jobs=-1,
        class_weight=cw,
        refit=True,
        random_state=random_state
    )
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler(with_mean=True, with_std=True)),
        ('clf', clf)
    ])

    pipe.fit(X_tr, y_tr)

    # predictions
    yprob_tr = pipe.predict_proba(X_tr)[:, 1]
    yhat_tr  = pipe.predict(X_tr)
    yprob_te = pipe.predict_proba(X_te)[:, 1]
    yhat_te  = pipe.predict(X_te)

    # metrics
    m_tr = eval_metrics(y_tr, yprob_tr, yhat_tr)
    m_te = eval_metrics(y_te, yprob_te, yhat_te)

    # coefficients (on standardized features)
    coefs = pipe.named_steps['clf'].coef_[0]
    coef_df = pd.DataFrame({'Feature': Xdf.columns, 'Coef': coefs}).sort_values('Coef', ascending=False)
    nnz = np.count_nonzero(coefs)
    print(f"  Non-zero coefficients: {nnz}")

    # save
    tag = f"LASSO_LIBLINEAR_NoSMOTE_{feature_set_name}_{class_weight_mode.upper()}"
    base = os.path.join(OUTDIR, tag)

    pd.DataFrame([{'FeatureSet': feature_set_name, 'Split': 'Train', **m_tr},
                  {'FeatureSet': feature_set_name, 'Split': 'Test',  **m_te}]).to_csv(base+"_Metrics.csv", index=False)

    pd.DataFrame({'Sample': tr_ids, 'true_class_(0/1)': y_tr,
                  'predicted_probability': yprob_tr, 'predicted_label': yhat_tr}).to_csv(base+"_Predictions_Train.csv", index=False)
    pd.DataFrame({'Sample': te_ids, 'true_class_(0/1)': y_te,
                  'predicted_probability': yprob_te, 'predicted_label': yhat_te}).to_csv(base+"_Predictions_Test.csv", index=False)

    coef_df.to_csv(base+"_Coefficients.csv", index=False)
    print(f"  ✓ Saved: {base}_Metrics.csv")
    print(f"  ✓ Saved: {base}_Predictions_Train.csv")
    print(f"  ✓ Saved: {base}_Predictions_Test.csv")
    print(f"  ✓ Saved: {base}_Coefficients.csv")
    return m_tr, m_te, nnz

# =========================
# Run (ALL_GENES & DEG_ONLY) with/without class weighting
# =========================
rows = []
for cw_mode in CLASS_WEIGHT_MODES:
    for name, X in [("ALL_GENES", X_all), ("DEG_ONLY", X_deg)]:
        mtr, mte, nnz = run_lasso_once(X, y, feature_set_name=name, class_weight_mode=cw_mode)
        rows.append({'FeatureSet': name, 'ClassWeight': cw_mode, 'Split': 'Train', **mtr, 'NonZero_Coefs': nnz})
        rows.append({'FeatureSet': name, 'ClassWeight': cw_mode, 'Split': 'Test',  **mte, 'NonZero_Coefs': nnz})

summary = pd.DataFrame(rows)
summary_path = os.path.join(OUTDIR, "LASSO_NoSMOTE_ALLvsDEG_BalancedVsNone_Summary.csv")
summary.to_csv(summary_path, index=False)
print(f"\n✓ Summary saved: {summary_path}")

# quick print of the key Test metrics
print("\n=== Test metrics (Accuracy, AUC, Sens/Spec, BalAcc) ===")
print(summary[summary['Split']=='Test'][[
    'FeatureSet','ClassWeight','Accuracy','ROC AUC','Sensitivity(Tumor)','Specificity(Tumor)','Recall(Normal)','Balanced Accuracy','NonZero_Coefs'
]])


▶ ALL_GENES | class_weight=balanced | n_samples=1,224 | n_features=40,967
  Non-zero coefficients: 195
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_ALL_GENES_BALANCED_Metrics.csv
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_ALL_GENES_BALANCED_Predictions_Train.csv
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_ALL_GENES_BALANCED_Predictions_Test.csv
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_ALL_GENES_BALANCED_Coefficients.csv

▶ DEG_ONLY | class_weight=balanced | n_samples=1,224 | n_features=5,737
  Non-zero coefficients: 126
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_DEG_ONLY_BALANCED_Metrics.csv
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_DEG_ONLY_BALANCED_Predictions_Train.csv
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_DEG_ONLY_BALANCED_Predictions_Test.csv
  ✓ Saved: BRCA_ML_Outputs/LASSO_LIBLINEAR_NoSMOTE_DEG_ONLY_BALANCED_Coefficients.csv

▶ ALL_GENES | class_weight=none | n_samples=1,224 | n_features=40,967
  Non-zero coefficients